In [6]:
import torch
import pickle
import random

from sklearn.decomposition import PCA



class close_words:
    def __init__(self,model,w2i,pca=False,n=5):
        self.model=model
        self.w2i=w2i
        self.n=n
        self.i2w=dict(zip(w2i.values(),w2i.keys()))
        pca = PCA(n_components=2)
        self.embeddings=model['embedding_in.weight'] if not pca else torch.tensor(pca.fit_transform(model['embedding_in.weight']))
        
        
    def input_word(self,word):
        self.sample_idx=self.w2i[word]
        self.sample_vec=self.embeddings[self.sample_idx]
    
    def sample_word(self):
        
        self.sample_idx=random.sample(list(self.w2i.values()),1)[0]
        self.sampled_word=self.i2w[self.sample_idx]
        self.sample_vec=self.embeddings[self.sample_idx]
        print('임의의 단어:',self.sampled_word)
    
    def l2_dist(self):
        
        self.trial=torch.mul((self.embeddings-self.sample_vec),(self.embeddings-self.sample_vec))
        self.trial[self.sample_idx]=100
        self.trial=torch.sum(self.trial,dim=1)
        #rec=torch.argmax(-self.trial).item()
        idexes=torch.argsort(-self.trial, descending=True)[:self.n]
        step=1
        print(len(self.i2w))
        for idx in idexes:
            if idx==len(w2i): #unk 제거
                continue
            print(step,'번째 가까운 단어:',self.i2w[idx.item()])
            step+=1
                
    def cos_sim(self):
      
        self.trial=torch.div(torch.sum(torch.mul(self.embeddings,self.sample_vec),dim=1),torch.sum(torch.mul(self.embeddings,self.embeddings),dim=1))
     
        self.trial[self.sample_idx]=-100
        idexes=torch.argsort(self.trial, descending=True)[:self.n]
        step=1
        for idx in idexes:
            if idx==len(w2i)+1: #unk 제거
                continue
            print(step,'번째 가까운 단어:',self.i2w[idx.item()])
            step+=1
        

In [103]:
#model=torch.load('./w2v_withoud_ns',map_location='cpu')
model=torch.load('./withous_gut_su_haha_epoch_3',map_location='cpu')['state_dict']

with open('./preprocessing//vocab_without_josa_gut_su.pickle','rb') as f:
    w2i=pickle.load(f)
    
print('임베딩 사이즈:',model['embedding_in.weight'].shape)  #unk 랑 embedding이 0부터인것을 몰랐음. 0은 버리면 됨
print('사전 사이즈:',len(w2i))  

임베딩 사이즈: torch.Size([28053, 256])
사전 사이즈: 41034


In [93]:
showing=close_words(model,w2i,pca=True)
showing.sample_word()
showing.l2_dist()
print('임의의 단어',showing.sampled_word)
showing.cos_sim()

임의의 단어: ('여덟', 'Noun')
28052
1 번째 가까운 단어: ('foreigners', 'Alpha')
2 번째 가까운 단어: ('탕', 'Noun')
3 번째 가까운 단어: ('빠르게', 'Adjective')
4 번째 가까운 단어: ('친절하십니다만', 'Adjective')
5 번째 가까운 단어: ('외우기', 'Verb')
임의의 단어 ('여덟', 'Noun')
1 번째 가까운 단어: ('빡치는건', 'Adjective')
2 번째 가까운 단어: ('아는데', 'Verb')
3 번째 가까운 단어: ('더욱이', 'Noun')
4 번째 가까운 단어: ('보나', 'Noun')
5 번째 가까운 단어: ('즐겁지만', 'Adjective')


In [101]:
showing.input_word(('버스','Noun'))
showing.l2_dist()
showing.cos_sim()

28052
1 번째 가까운 단어: ('메데이아', 'Noun')
2 번째 가까운 단어: ('충분해서', 'Adjective')
3 번째 가까운 단어: ('떨어지는게', 'Verb')
4 번째 가까운 단어: ('비제', 'Noun')
5 번째 가까운 단어: ('편할듯', 'Adjective')
1 번째 가까운 단어: ('들으시기', 'Verb')
2 번째 가까운 단어: ('빡치는건', 'Adjective')
3 번째 가까운 단어: ('황금', 'Noun')
4 번째 가까운 단어: ('싫으', 'Adjective')
5 번째 가까운 단어: ('쿼터', 'Noun')


## UNK 빈도수

In [98]:
with open('./preprocessing/vocab_without_josa_gut_su_freq.pickle','rb') as f:
    total_freq=pickle.load(f)
    
with open('./preprocessing/vocab_without_josa_gut_su.pickle','rb') as h:
    voc=pickle.load(h)
    
not_unk=list(voc.keys())

freq_sum=0

def summing(x):
    
    global freq_sum
    freq_sum+=x

_=list(map(lambda x:summing(total_freq[x]),not_unk))

ratio=(sum(total_freq.values())-freq_sum)/sum(total_freq.values())
print(ratio)

0.014246669941204219


In [102]:
## unk 빈도수가 문제는 아닌듯??